# Dijkstra's Algorithm: Essential Notes

## Introduction and Context

### SSSP Algorithm Family

| Graph Type | Weight Restrictions | Algorithm | Time Complexity |
|------------|-------------------|-----------|-----------------|
| General | Unweighted | BFS | $O(\|V\| + \|E\|)$ |
| DAG | Any weights | DAG Relaxation | $O(\|V\| + \|E\|)$ |
| General | **Non-negative** | **Dijkstra** | $O(\|V\| \log \|V\| + \|E\|)$ |
| General | Any weights | Bellman-Ford | $O(\|V\| \cdot \|E\|)$ |

**Key Point:** Dijkstra is optimal for weighted graphs with **non-negative edge weights**.

### Why Dijkstra?

BFS fails on weighted graphs. In this example, BFS says shortest path from $s$ to $d$ is direct (1 edge), but actual shortest is $s \rightarrow a \rightarrow b \rightarrow d$:

```
s ——————100——————→ d
|                 ↑
1                 1
↓                 |
a ————1———→ b ————
```

## Core Mathematical Concepts

### Non-negative Weights Property

**Key Insight:** If all edge weights are non-negative, distances are **monotonic** along shortest paths.

**Mathematical Statement:** If vertex $u$ appears on shortest path from $s$ to $v$:
$$\delta(s, u) \leq \delta(s, v)$$

This allows processing vertices in order of increasing distance.

### Relaxation Operation

For edge $(u, v)$ with weight $w(u, v)$:

$$\text{if } d(s, v) > d(s, u) + w(u, v) \text{ then } d(s, v) := d(s, u) + w(u, v)$$

**Meaning:** "Can I reach $v$ faster by going through $u$?"

## The Algorithm

### Core Idea
1. **Process vertices in distance order** from source
2. **Use priority queue** to efficiently find next closest vertex
3. **Relax edges** to update distance estimates

### Data Structure: Changeable Priority Queue

**Definition:** A priority queue $Q$ that supports:

| Operation | Purpose | Time Complexity |
|-----------|---------|-----------------|
| `Q.build(X)` | Initialize $Q$ with items from iterator $X$ | $O(n)$ |
| `Q.delete_min()` | Remove and return item with minimum key | $O(\log n)$ |
| `Q.decrease_key(id, k)` | Update item with ID `id` to have key `k` | $O(\log n)$ |

**Implementation Details:**
- **Cross-linking:** Combine Priority Queue $Q'$ with Dictionary $D$ mapping IDs to positions in $Q'$
- **Vertex IDs:** Assume integers from $0$ to $|V| - 1$ for direct array access
- **Item representation:** Each item is tuple $(v, d(s, v))$ where $v$ is vertex ID and $d(s, v)$ is distance estimate

### Operation Count

Dijkstra performs these operations on a priority queue with $n$ items:

| Operation | Times Called | Individual Cost |
|-----------|-------------|-----------------|
| `Q.build(X)` | $1$ | $B_n$ |
| `Q.delete_min()` | $|V|$ | $M_n$ |
| `Q.decrease_key()` | $|E|$ | $D_n$ |

**Total Time:** $O(B_{|V|} + |V| \cdot M_{|V|} + |E| \cdot D_{|V|})$

### Detailed Algorithm Steps

**Initialization:**
1. Set $d(s, v) = \infty$ for all $v \in V$
2. Set $d(s, s) = 0$ 
3. Build priority queue $Q$ with items $(v, d(s, v))$ for each vertex $v$

**Main Loop:**
4. While $Q$ is not empty:
   a. Delete item $(u, d(s, u))$ with minimum $d(s, u)$ from $Q$
   b. For each vertex $v$ in outgoing adjacencies $\text{Adj}^+(u)$:
      i. If $d(s, v) > d(s, u) + w(u, v)$:
         - Relax: set $d(s, v) = d(s, u) + w(u, v)$  
         - Decrease key of $v$ in $Q$ to new $d(s, v)$

## Example Trace

### Initial Setup

![Graph](./07-1-graph.png)


### Step-by-Step Execution

![table1](./07-2.png)
![table2](./07-3.png)
![table3](./07-4.png)
![table4](./07-5.png)
![table5](./07-6.png)



## Correctness Proof

**Theorem:** At algorithm end, $d(s, v) = \delta(s, v)$ for all $v \in V$.

**Proof by induction** on vertices removed from queue:

**Base:** First vertex removed is $s$ with $d(s, s) = 0 = \delta(s, s)$ ✓

**Inductive Step:** For $k_0$-th vertex $v_0$ removed, consider shortest path $\pi$ from $s$ to $v_0$. Let $(x, y)$ be first edge where $y$ wasn't among first $k_0-1$ vertices.

When $x$ was removed: $d(s, y) \leq \delta(s, x) + w(x, y) = \delta(s, y) \leq \delta(s, v_0) \leq d(s, v_0) \leq d(s, y)$

All inequalities are equalities, so $d(s, v_0) = \delta(s, v_0)$ ✓

## Time Complexity

### Implementation Choices

| Priority Queue | Build | Delete Min | Decrease Key | Total Time | Best For |
|---------------|-------|------------|--------------|------------|----------|
| Array | $O(n)$ | $O(n)$ | $O(1)$ | $O(\|V\|^2)$ | Dense graphs |
| Binary Heap | $O(n)$ | $O(\log n)$ | $O(\log n)$ | $O(\|E\| \log \|V\|)$ | Sparse graphs |
| Fibonacci Heap | $O(n)$ | $O(\log n)$ | $O(1)$ | $O(\|E\| + \|V\| \log \|V\|)$ | Theoretical optimum |

### Practical Recommendations

**For Dense Graphs:** $|E| = \Theta(|V|^2)$ → Use Array implementation → $O(|V|^2)$ time

**For Sparse Graphs:** $|E| = \Theta(|V|)$ → Use Binary Heap → $O(|V| \log |V|)$ time

**General Assumption:** When analyzing algorithms theoretically, assume Dijkstra runs in $O(|E| + |V| \log |V|)$ time.

## Implementation

```python
import heapq

def dijkstra(graph, start):
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, u = heapq.heappop(pq)
        
        if u in visited:
            continue
        visited.add(u)
        
        for v, weight in graph[u]:
            if v not in visited:
                new_dist = current_dist + weight
                if new_dist < distances[v]:
                    distances[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))
    
    return distances

import heapq

// OR

from collections import defaultdict

def dijkstra(graph, start):
    """
    Dijkstra's algorithm implementation
    
    Args:
        graph: dict where graph[u] = [(v, weight), ...]
        start: starting vertex
    
    Returns:
        distances: dict of shortest distances from start
    """
    # Initialize distances
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    
    # Priority queue: (distance, vertex)
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, u = heapq.heappop(pq)
        
        # Skip if already processed
        if u in visited:
            continue
            
        visited.add(u)
        
        # Relax edges
        for v, weight in graph[u]:
            if v not in visited:
                new_dist = current_dist + weight
                if new_dist < distances[v]:
                    distances[v] = new_dist
                    heapq.heappush(pq, (new_dist, v))
    
    return distances
```

## When to Use Which Algorithm

### Decision Tree: Which Algorithm to Use?

```
Is your graph weighted?
├── No → Use BFS
└── Yes → Are there negative edge weights?
    ├── Yes → Use Bellman-Ford  
    └── No → Is it a DAG?
        ├── Yes → Use DAG Relaxation
        └── No → Use Dijkstra
```

## Key Applications

1. **GPS Navigation** - Find shortest/fastest routes
2. **Network Routing** - Optimal data transmission paths  
3. **Social Networks** - Degrees of separation

## Common Mistakes

❌ **Don't use with negative weights** - gives wrong answers!
❌ **Don't forget visited check** - may process vertices multiple times

✅ **Always verify non-negative weights** before using
✅ **Choose right data structure** based on graph density

## Summary

**Use Dijkstra when:**
- Weighted graph with non-negative edges
- Need single-source shortest paths
- Want optimal performance for this case

**Key insight:** Greedy approach works because distances increase monotonically along shortest paths.

**Time complexity:** $O(\|E\| + \|V\| \log \|V\|)$ with proper implementation.

# Dijkstra's Algorithm: Interview Preparation Guide

## 🎯 Interview Likelihood: VERY HIGH

Dijkstra's algorithm is **extremely common** in technical interviews, especially at:
- **FAANG companies** (Google, Meta, Amazon, Microsoft, Apple)
- **Tech unicorns** (Uber, Airbnb, ByteDance)
- **Finance/Trading firms** (Two Sigma, Citadel, Jane Street)
- **All levels** from new grad to senior engineer

## 📊 Common Interview Question Types

### 1. **Direct Implementation** (60% probability)
- "Implement Dijkstra's algorithm"
- "Find shortest path in weighted graph"
- "Return shortest distances from source to all vertices"

### 2. **Disguised Problems** (30% probability)
- "Cheapest flights with K stops" (Leetcode 787)
- "Network delay time" (Leetcode 743)
- "Path with minimum effort" (Leetcode 1631)
- "Swim in rising water" (Leetcode 778)

### 3. **Comparison Questions** (10% probability)
- "When would you use Dijkstra vs BFS vs Bellman-Ford?"
- "What's the time complexity and why?"
- "How does this compare to A* algorithm?"

## 🗣️ Perfect Interview Responses

### Question 1: "Explain Dijkstra's algorithm"

**Perfect Answer Structure:**
```
"Dijkstra's algorithm finds shortest paths from a source vertex to all other 
vertices in a weighted graph with non-negative edge weights.

Key insight: We can process vertices in order of increasing distance from 
source because of the non-negative weight property - distances are monotonic 
along shortest paths.

The algorithm uses a priority queue to efficiently find the next closest 
unprocessed vertex, then relaxes all edges from that vertex to update 
distance estimates.

Time complexity is O(E + V log V) with a binary heap implementation."
```

**Follow-up:** "Why does it fail with negative weights?"
**Answer:** "With negative weights, a longer path (more edges) could have smaller total weight, violating our assumption that we can finalize distances when processing vertices in order."

### Question 2: "Implement Dijkstra's algorithm"

**What to say before coding:**
```
"I'll implement using a min-heap priority queue. The key steps are:
1. Initialize distances with infinity except source = 0
2. Use priority queue to always process nearest unvisited vertex
3. Relax edges to update distance estimates
4. Track visited vertices to avoid reprocessing"
```

**Code to write:**
```python
import heapq

def dijkstra(graph, start):
    # Initialize distances
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Priority queue: (distance, node)
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, current = heapq.heappop(pq)
        
        # Skip if already processed
        if current in visited:
            continue
            
        visited.add(current)
        
        # Relax edges
        for neighbor, weight in graph[current]:
            if neighbor not in visited:
                new_dist = current_dist + weight
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    heapq.heappush(pq, (new_dist, neighbor))
    
    return distances

# Example usage
graph = {
    'A': [('B', 4), ('C', 2)],
    'B': [('C', 1), ('D', 5)],
    'C': [('D', 8), ('E', 10)],
    'D': [('E', 2)],
    'E': []
}

result = dijkstra(graph, 'A')
print(result)  # {'A': 0, 'B': 4, 'C': 2, 'D': 9, 'E': 11}
```

**Key points to mention:**
- "I'm using heapq for the priority queue implementation"
- "The visited set prevents reprocessing vertices"
- "We only push to heap when we find a better distance"

### Question 3: "Find shortest path between two specific nodes"

**Modified implementation:**
```python
def dijkstra_shortest_path(graph, start, end):
    distances = {node: float('inf') for node in graph}
    previous = {node: None for node in graph}
    distances[start] = 0
    
    pq = [(0, start)]
    visited = set()
    
    while pq:
        current_dist, current = heapq.heappop(pq)
        
        # Early termination when we reach target
        if current == end:
            break
            
        if current in visited:
            continue
            
        visited.add(current)
        
        for neighbor, weight in graph[current]:
            if neighbor not in visited:
                new_dist = current_dist + weight
                if new_dist < distances[neighbor]:
                    distances[neighbor] = new_dist
                    previous[neighbor] = current
                    heapq.heappush(pq, (new_dist, neighbor))
    
    # Reconstruct path
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]
    path.reverse()
    
    return distances[end], path if distances[end] != float('inf') else (float('inf'), [])
```

**Key optimization to mention:** "I added early termination when we reach the target, and path reconstruction using a previous pointer array."

## 🔥 Advanced Interview Questions

### Question 4: "Analyze time complexity for different implementations"

**Framework Answer:**
```
"Time complexity depends on the priority queue implementation:

1. Binary Heap: O(E log V)
   - Each of E edges might trigger decrease-key operation: O(log V)
   - Each of V vertices extracted once: O(V log V)
   - Total: O((E + V) log V) = O(E log V) for connected graphs

2. Array (for dense graphs): O(V²)
   - Extract min: O(V) done V times = O(V²)
   - Decrease key: O(1) done E times = O(E)
   - Better when E = Θ(V²)

3. Fibonacci Heap (theoretical): O(E + V log V)
   - Amortized decrease-key: O(1)
   - But complex implementation, rarely used in practice

For interviews, assume O(E log V) with binary heap."
```

### Question 5: "When would you NOT use Dijkstra?"

**Perfect Answer:**
```
"Don't use Dijkstra when:

1. Negative edge weights exist → Use Bellman-Ford
2. Unweighted graph → Use BFS (simpler and faster)
3. Need to detect negative cycles → Use Bellman-Ford
4. Graph is a DAG → Use topological sort + relaxation (O(V + E))
5. Memory is extremely limited → Consider A* with good heuristic

Key insight: Dijkstra assumes non-negative weights for its greedy approach to work."
```

### Question 6: "How would you modify Dijkstra for these scenarios?"

**Scenario A: "Find K shortest paths"**
```python
def k_shortest_paths(graph, start, end, k):
    # Use modified Dijkstra where each node can be visited k times
    # Priority queue stores (distance, node, path_count)
    # This is actually Yen's algorithm territory - mention it's complex
    pass
```

**Scenario B: "Graph changes during execution"**
```python
def dynamic_dijkstra(graph, start, updates):
    # Mention this becomes complex - need D* algorithm
    # For interview: "I'd rerun Dijkstra after each update"
    pass
```

## 🧠 LeetCode Problems to Master

### Easy/Medium (Must Know):
1. **Network Delay Time** (LC 743) - Direct Dijkstra application
2. **Cheapest Flights Within K Stops** (LC 787) - Modified Dijkstra
3. **Path With Minimum Effort** (LC 1631) - Binary search + Dijkstra

### Hard (Bonus Points):
1. **Swim in Rising Water** (LC 778) - Binary search + pathfinding
2. **Minimum Cost to Make at Least One Valid Path** (LC 1368)

### Template for Network Delay Time:
```python
def networkDelayTime(times, n, k):
    # Build graph
    graph = defaultdict(list)
    for u, v, w in times:
        graph[u].append((v, w))
    
    # Run Dijkstra
    distances = dijkstra(graph, k)
    
    # Check if all nodes reachable
    max_dist = 0
    for i in range(1, n + 1):
        if i not in distances or distances[i] == float('inf'):
            return -1
        max_dist = max(max_dist, distances[i])
    
    return max_dist
```

## ⚠️ Common Interview Mistakes

### ❌ Red Flags to Avoid:
1. **Forgetting the visited set** → May process vertices multiple times
2. **Using with negative weights** → Will give wrong answers
3. **Not handling disconnected graphs** → Some nodes stay at infinity
4. **Wrong priority queue usage** → Using max-heap instead of min-heap
5. **Inefficient graph representation** → Using adjacency matrix for sparse graphs

### ✅ Green Flags to Hit:
1. **Mention non-negative weight requirement** upfront
2. **Explain the greedy choice property**
3. **Handle edge cases** (empty graph, single node, unreachable nodes)
4. **Optimize when possible** (early termination for single target)
5. **Analyze time/space complexity** correctly

## 🎪 Behavioral Integration

### "Tell me about a time you optimized an algorithm"

**Structure your answer:**
```
"I was working on a route optimization service where we needed fastest paths 
in a road network.

**Situation**: Initial BFS approach failed because roads have different speeds
**Task**: Find actual fastest routes considering travel times
**Action**: Implemented Dijkstra's algorithm with travel time as edge weights,
          optimized with early termination when reaching destination
**Result**: Reduced average query time by 60% and gave correct shortest paths

The key insight was recognizing this as a single-source shortest path problem
with non-negative weights, making Dijkstra the perfect fit."
```

